In [129]:
import os
import json

In [251]:
dataset_name = "mvtec_ad_bottle"

dataset_coco = f"/home/nisyad/projects/industrial-defect-detection/datasets/processed/{dataset_name}/test.json"

# Zero-shot responses
responses = f"/home/nisyad/projects/industrial-defect-detection/{dataset_name}_responses.json"

# 1-shot In-context responses
# responses = f"/home/nisyad/projects/industrial-defect-detection/{dataset_name}_responses_in_context.json"

In [252]:
with open(dataset_coco, "r") as f:
    coco = json.load(f)
    
categories = coco["categories"]
categories


[{'id': 1, 'name': 'good'},
 {'id': 2, 'name': 'broken'},
 {'id': 3, 'name': 'contamination'}]

In [253]:
with open(responses, "r") as f:
    results = json.load(f)
    
results[:3]

[{'sample_idx': 0,
  'response': '```json\n{\n  "is_defective": "yes",\n  "reason": "The bottle has contamination on its inside surface.",\n  "defect_type": "contamination",\n  "bounding_box": "[0.0, 0.2, 0.2, 0.5]"\n}\n```',
  'target': [[0.0, -1.0, -1.0, -1.0, -1.0]],
  'parsed_response': {'is_defective': 'yes',
   'reason': 'The bottle has contamination on its inside surface.',
   'defect_type': 'contamination',
   'bounding_box': '[0.0, 0.2, 0.2, 0.5]'}},
 {'sample_idx': 1,
  'response': '```json\n{\n  "is_defective": "no",\n  "reason": "",\n  "defect_type": "",\n  "bounding_box": ""\n}\n```',
  'target': [[0.0, -1.0, -1.0, -1.0, -1.0]],
  'parsed_response': {'is_defective': 'no',
   'reason': '',
   'defect_type': '',
   'bounding_box': ''}},
 {'sample_idx': 2,
  'response': '{\n    "is_defective": "no",\n    "reason": "",\n    "defect_type": "",\n    "bounding_box": ""\n}',
  'target': [[0.0, -1.0, -1.0, -1.0, -1.0]],
  'parsed_response': {'is_defective': 'no',
   'reason': '',
 

In [254]:
is_defective_types = set()
is_defective_count = {"yes": 0, "no": 0, "other": 0}

for result in results:
    is_defective_type = result["parsed_response"]["is_defective"]
    if is_defective_type not in is_defective_count:
        is_defective_count["other"] += 1
    else:
        is_defective_count[is_defective_type] += 1
    is_defective_types.add(is_defective_type)
    

is_defective_types
print(is_defective_count)

{'yes': 74, 'no': 9, 'other': 0}


In [255]:
defect_types = set()

for result in results:
    defect_type = result["parsed_response"]["defect_type"]
    defect_types.add(defect_type)

defect_types

{'', 'broken', 'contamination'}

In [256]:
defect_types = list(defect_types)
defect_types

['contamination', '', 'broken']

In [257]:
# Split multiple defect types
defect_types = [defect_type.split(", ") for defect_type in defect_types]
defect_types

[['contamination'], [''], ['broken']]

In [258]:
def calculate_iou(box1, box2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters:
    - box1, box2: Lists of 4 coordinates [x_top, y_top, x_bottom, y_bottom].

    Returns:
    - IoU as a float value.
    """
    # Unpack the coordinates
    x1_top, y1_top, x1_bottom, y1_bottom = box1
    x2_top, y2_top, x2_bottom, y2_bottom = box2

    # Calculate the (x, y)-coordinates of the intersection rectangle
    x_inter_top = max(x1_top, x2_top)
    y_inter_top = max(y1_top, y2_top)
    x_inter_bottom = min(x1_bottom, x2_bottom)
    y_inter_bottom = min(y1_bottom, y2_bottom)

    # Compute the area of intersection rectangle
    inter_area = max(0, x_inter_bottom - x_inter_top) * max(0, y_inter_bottom - y_inter_top)

    # Compute the area of both the prediction and ground-truth rectangles
    box1_area = (x1_bottom - x1_top) * (y1_bottom - y1_top)
    box2_area = (x2_bottom - x2_top) * (y2_bottom - y2_top)

    # Compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = inter_area / float(box1_area + box2_area - inter_area)

    return iou

In [259]:
# Calculate the mean IoU

total_iou = 0

for result in results:
    is_defective = result["parsed_response"]["is_defective"]
    target = result["target"][0][0]
     
    if (is_defective == "yes" and target != 0):
        tgt_bbox = result["target"][0][1:]
        pred_bbox = result["parsed_response"]["bounding_box"]
        
        try:
            pred_bbox = json.loads(pred_bbox) if not isinstance(pred_bbox, list) else pred_bbox
        except:
            pred_bbox = [0, 0, 0, 0]
        iou = calculate_iou(tgt_bbox, pred_bbox)
        total_iou += iou
        
    if (is_defective == "no" and target == 0):
        total_iou += 1
    
mean_iou = total_iou / len(results)
print(mean_iou)

0.2179461145830379


In [260]:
def calculate_binary_accuracy(responses):
    not_defective_correct = 0
    defective_correct = 0
    for item in responses:
        response = item["parsed_response"]
        target = item["target"][0][0]
        
        # print(response["is_defective"], target)
        
        if target == 0 and response["is_defective"] == "no":
            not_defective_correct += 1
            # print("non defective correct: ", not_defective_correct)
        elif target != 0.0 and response["is_defective"] == "yes":
            defective_correct += 1
            # print("defective correct: ", defective_correct)
            
    print(not_defective_correct, defective_correct)
    return (not_defective_correct + defective_correct )/ len(responses)


print(len(results))
calculate_binary_accuracy(results)
   

83
6 60


0.7951807228915663

In [261]:
results

[{'sample_idx': 0,
  'response': '```json\n{\n  "is_defective": "yes",\n  "reason": "The bottle has contamination on its inside surface.",\n  "defect_type": "contamination",\n  "bounding_box": "[0.0, 0.2, 0.2, 0.5]"\n}\n```',
  'target': [[0.0, -1.0, -1.0, -1.0, -1.0]],
  'parsed_response': {'is_defective': 'yes',
   'reason': 'The bottle has contamination on its inside surface.',
   'defect_type': 'contamination',
   'bounding_box': '[0.0, 0.2, 0.2, 0.5]'}},
 {'sample_idx': 1,
  'response': '```json\n{\n  "is_defective": "no",\n  "reason": "",\n  "defect_type": "",\n  "bounding_box": ""\n}\n```',
  'target': [[0.0, -1.0, -1.0, -1.0, -1.0]],
  'parsed_response': {'is_defective': 'no',
   'reason': '',
   'defect_type': '',
   'bounding_box': ''}},
 {'sample_idx': 2,
  'response': '{\n    "is_defective": "no",\n    "reason": "",\n    "defect_type": "",\n    "bounding_box": ""\n}',
  'target': [[0.0, -1.0, -1.0, -1.0, -1.0]],
  'parsed_response': {'is_defective': 'no',
   'reason': '',
 

In [247]:
# Calculate multiclass accuracy - transistor

good = 0
broken_or_misplaced_lead = 0
misplaced_transistor = 0
damaged_casing = 0

for result in results:
    defect_type = result["parsed_response"]["defect_type"]
    # print(defect_type)
    target = result["target"][0][0]
    # print(target)
    
    if defect_type == "" and target == 0:
        good += 1
    elif defect_type in ["broken_or_misplaced lead", "broken_or_misplaced_lead" ] and (target == 1):
        broken_or_misplaced_lead += 1
    elif defect_type == "misplaced transistor" and target == 2:
        misplaced_transistor += 1
    elif defect_type == "damaged casing" and target == 3:
        damaged_casing += 1
        
    
    
print(good, broken_or_misplaced_lead, misplaced_transistor, damaged_casing)
print((good + broken_or_misplaced_lead + misplaced_transistor + damaged_casing) / len(results))

19 0 0 0
0.11875


In [248]:
# # Calculate multiclass accuracy - screw

good = 0
damaged_head = 0
damaged_shank = 0
damaged_thread = 0
damaged_tip = 0

for result in results:
    defect_type = result["parsed_response"]["defect_type"]
    # print(defect_type)
    defect_types = [defect_type.split(", ")]
    # print(defect_types)
    target = result["target"][0][0]
    # print(target)
    
    # print(defect_types, target)
    
    for defect_type in defect_types[0]:
        if defect_type == "" and target == 0:
            good += 1
        elif defect_type == "damaged head" and (target == 1):
            damaged_head += 1
        elif defect_type == "damaged shank" and target == 2:
            damaged_shank += 1
        elif defect_type in [ "damaged thread", "damaged-thread" ] and target == 3:
            damaged_thread += 1
        elif defect_type == "damaged tip" and target == 4:
            damaged_tip += 1
        
    
    
print(good, damaged_head, damaged_shank, damaged_thread, damaged_tip)
print((good + damaged_head + damaged_shank + damaged_thread + damaged_tip) / len(results))

19 18 22 33 23
0.71875


In [263]:
# # Calculate multiclass accuracy - bottle

good = 0
broken = 0
contamination = 0

for result in results:
    defect_type = result["parsed_response"]["defect_type"]
    # print(defect_type)
    defect_types = [defect_type.split(", ")]
    # print(defect_types)
    target = result["target"][0][0]
    
    for defect_type in defect_types[0]:
        if defect_type == "" and target == 0:
            good += 1
        elif defect_type == "broken" and (target == 1):
            broken += 1
        elif defect_type == "contamination" and target == 2:
            contamination += 1
        
print(good, broken, contamination)
print((good + broken + contamination) / len(results))

6 33 14
0.6385542168674698


In [250]:
# # Calculate multiclass accuracy - metal_plate

good = 0
rusting = 0
scratches = 0

for result in results:
    defect_type = result["parsed_response"]["defect_type"]
    # print(defect_type)
    defect_types = [defect_type.split(", ")]
    # print(defect_types)
    target = result["target"][0][0]
   
    
    for defect_type in defect_types[0]:
        if defect_type == "" and target == 0:
            good += 1
        elif defect_type == "rusting" and (target == 1):
            rusting += 1
        elif defect_type == "scratches" and target == 2:
            scratches += 1
        
print(good, rusting, scratches)
print((good + rusting + scratches) / len(results))

19 0 0
0.11875
